# Título  Data quality y drifting
## Autor: Jose Chelquer
## Fecha de última modificación: 20/11/2024

Toma el archivo que ya tiene la clase ternaria y le hace el control de data quality y drifting


## Parámetros

< Descripción de cada uno de los parámetros que utiliza el job >


In [21]:
vm=True
if vm:
  usar_gdrive=False
else:
  usar_gdrive=True


In [22]:
experimento="____"  # partiendo de la base
tipo_registros=experimento[0]            # '_'completo s sampleado
con_preprocesamiento = experimento[1]    # p con preprocesamiento  '_' sin preprocesamiento
tipo_featuring=experimento[2]            # _: 155 base  s: simple    c: completo
rf=experimento[3]                        # _: sin RF   r: con RF

dict_tipo_registros={'_': 'completo', 's': 'sampleado'}
dict_con_preprocesamiento={'_': 'sin preprocesamiento', 'p':'con preprocesamiento'}
dict_tipo_featuring={'_': 'base', 's':' simple', 'c': 'completo'}
dict_rf={'_': 'sin RF', 'f':' con RF'}

print (f'Experimento {experimento}')
print (dict_tipo_registros[tipo_registros])
print (dict_con_preprocesamiento[con_preprocesamiento])
print (dict_tipo_featuring[tipo_featuring])
print (dict_rf[rf])

Experimento ____
completo
sin preprocesamiento
base
sin RF


## Input

In [23]:
if vm:
  dataset_path = '~/buckets/b1/datasets'
  
else:
  dataset_path='/content/drive/MyDrive/Data Science y similares/Maestría Data Mining Exactas/dmeyf/dmeyf2024/datasets/'

dataset_file=f'k3_{experimento}.parquet.gz'
    
print (dataset_path)
print (dataset_file)

~/buckets/b1/datasets
k3_____.parquet.gz


## Output

< Archivos, bases de datos, modelos que va a generar el job>

In [24]:
output_file=f'k3_{tipo_registros}p{tipo_featuring}{rf}.parquet.gz'
print (output_file)

k3__p__.parquet.gz


## Procesos

### Paquetes necesarios

## Código del proceso

< Todo el código a partir de aquí debe poder ejecutarse sin necesidad de parametrizar nada>

In [25]:
if usar_gdrive:
  from google.colab import drive
  drive.mount('/content/drive')

## Librerías

In [26]:
!pip install --upgrade pip
!pip install --upgrade lightgbm
!pip install dask[dataframe]

In [27]:
#Librerías.
import pandas as pd
import numpy as np
import os


## Lectura de datos

In [28]:
#Lectura de datos.
dataset_pf=os.path.join(dataset_path, dataset_file)
data = pd.read_parquet(dataset_pf)

In [29]:
data.shape

(4901237, 155)

In [30]:
data['foto_mes'].value_counts()

foto_mes
202109    165644
202108    165442
202107    165152
202106    164876
202105    164623
202104    164090
202103    163685
202102    162646
202101    162026
202012    161526
202011    160742
202010    159731
202009    158371
202008    157058
202007    155764
202006    153757
202005    151261
202004    149872
202003    149356
202002    147109
202001    143966
201912    140661
201911    138667
201910    136682
201909    134314
201908    132664
201907    130724
201906    129186
201905    127659
201904    126996
201903    126436
201902    125799
201901    124752
Name: count, dtype: int64

## B. Data Quality.

#### 1. Valores nulos.

In [31]:
nulos_por_mes = data.groupby('foto_mes').apply(lambda x: x.isnull().sum())
nulos_porcentaje = data.groupby('foto_mes').apply(lambda x: (x.isnull().sum() / len(x)) * 100)

# Calcular la mediana de nulos para cada columna
mediana_nulos = nulos_porcentaje.median()

umbral=1
# Definir el umbral como 100% mayor o menor que la mediana
umbral_superior = mediana_nulos * (1+umbral)
umbral_inferior = mediana_nulos * (1-umbral)

# Identificar valores atípicos
valores_atipicos = nulos_porcentaje.apply(lambda fila: (fila > umbral_superior) | (fila < umbral_inferior), axis=1)

# Filtrar las columnas y meses con valores atípicos
resultados_atipicos = valores_atipicos[valores_atipicos.any(axis=1)]

# Mostrar individualmente
# Recorrer cada fila en los resultados atípicos
for mes, fila in resultados_atipicos.iterrows():
    # Identificar las columnas con valores atípicos (True), excluyendo 'foto_mes'
    columnas_atipicas = fila[fila.index != 'foto_mes'][fila].index.tolist()
    # Imprimir el mes y las columnas atípicas
    print(f"Mes: {mes}, Columnas atípicas: {columnas_atipicas}")
    for columna in columnas_atipicas:
      print (columna)
      print (nulos_porcentaje[columna])


/tmp/ipykernel_2178/4147783882.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  nulos_por_mes = data.groupby('foto_mes').apply(lambda x: x.isnull().sum())


Mes: 201901, Columnas atípicas: ['tmobile_app', 'cmobile_app_trx']
tmobile_app
foto_mes
201901    100.0
201902    100.0
201903    100.0
201904    100.0
201905    100.0
201906    100.0
201907      0.0
201908      0.0
201909      0.0
201910      0.0
201911      0.0
201912      0.0
202001      0.0
202002      0.0
202003      0.0
202004      0.0
202005      0.0
202006      0.0
202007      0.0
202008      0.0
202009      0.0
202010      0.0
202011      0.0
202012      0.0
202101      0.0
202102      0.0
202103      0.0
202104      0.0
202105      0.0
202106      0.0
202107      0.0
202108      0.0
202109      0.0
Name: tmobile_app, dtype: float64
cmobile_app_trx
foto_mes
201901    100.0
201902    100.0
201903    100.0
201904    100.0
201905    100.0
201906    100.0
201907      0.0
201908      0.0
201909      0.0
201910      0.0
201911      0.0
201912      0.0
202001      0.0
202002      0.0
202003      0.0
202004      0.0
202005      0.0
202006      0.0
202007      0.0
202008      0.0
20200

/tmp/ipykernel_2178/4147783882.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  nulos_porcentaje = data.groupby('foto_mes').apply(lambda x: (x.isnull().sum() / len(x)) * 100)


#### 2. Valores 0.

In [32]:
ceros_por_mes = data.groupby('foto_mes').apply(lambda x: (x==0).sum())
ceros_porcentaje = data.groupby('foto_mes').apply(lambda x:(x==0).sum() / len(x) * 100)

# Calcular la mediana de ceros para cada columna
mediana_ceros = ceros_porcentaje.median()

umbral=1
# Definir el umbral como 100% mayor o menor que la mediana
umbral_superior = mediana_ceros * (1+umbral)
umbral_inferior = mediana_ceros * (1-umbral)

# Identificar valores atípicos
valores_atipicos = ceros_porcentaje.apply(lambda fila: (fila > umbral_superior) | (fila < umbral_inferior), axis=1)

# Filtrar las columnas y meses con valores atípicos
resultados_atipicos = valores_atipicos[valores_atipicos.any(axis=1)]

# Mostrar individualmente
# Recorrer cada fila en los resultados atípicos
atipicas_todas=set()
for mes, fila in resultados_atipicos.iterrows():
    # Identificar las columnas con valores atípicos (True), excluyendo 'foto_mes'
    columnas_atipicas = fila[fila.index != 'foto_mes'][fila].index.tolist()
    atipicas_todas = atipicas_todas | set(columnas_atipicas)
    # Imprimir el mes y las columnas atípicas
    print(f"Mes: {mes}, Columnas atípicas: {columnas_atipicas}")
    #for columna in columnas_atipicas:
    #  print (columna)
    #  print (ceros_porcentaje[columna])
print ("\n\n Resumen de columnas atipicas")

atipicas_todas



/tmp/ipykernel_2178/2208825846.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ceros_por_mes = data.groupby('foto_mes').apply(lambda x: (x==0).sum())


Mes: 201901, Columnas atípicas: ['ctransferencias_recibidas', 'mtransferencias_recibidas', 'chomebanking_transacciones', 'Master_mfinanciacion_limite', 'Visa_mfinanciacion_limite']
Mes: 201902, Columnas atípicas: ['ctransferencias_recibidas', 'mtransferencias_recibidas', 'Master_mfinanciacion_limite', 'Visa_mfinanciacion_limite']
Mes: 201903, Columnas atípicas: ['ctransferencias_recibidas', 'mtransferencias_recibidas', 'Master_mfinanciacion_limite', 'Visa_mfinanciacion_limite', 'Visa_Finiciomora']
Mes: 201904, Columnas atípicas: ['ctarjeta_visa_debitos_automaticos', 'mttarjeta_visa_debitos_automaticos', 'ctransferencias_recibidas', 'mtransferencias_recibidas', 'chomebanking_transacciones', 'Master_mfinanciacion_limite', 'Visa_mfinanciacion_limite']
Mes: 201905, Columnas atípicas: ['mrentabilidad', 'mrentabilidad_annual', 'mcomisiones', 'mactivos_margen', 'mpasivos_margen', 'ccomisiones_otras', 'mcomisiones_otras', 'ctransferencias_recibidas', 'mtransferencias_recibidas', 'chomebanking_

/tmp/ipykernel_2178/2208825846.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ceros_porcentaje = data.groupby('foto_mes').apply(lambda x:(x==0).sum() / len(x) * 100)


{'Master_Finiciomora',
 'Master_fultimo_cierre',
 'Master_mfinanciacion_limite',
 'Visa_Finiciomora',
 'Visa_fultimo_cierre',
 'Visa_mfinanciacion_limite',
 'active_quarter',
 'ccomisiones_otras',
 'cextraccion_autoservicio',
 'chomebanking_transacciones',
 'cliente_edad',
 'cmobile_app_trx',
 'ctarjeta_debito_transacciones',
 'ctarjeta_visa_debitos_automaticos',
 'ctarjeta_visa_transacciones',
 'ctransferencias_recibidas',
 'internet',
 'mactivos_margen',
 'mautoservicio',
 'mcomisiones',
 'mcomisiones_otras',
 'mcuentas_saldo',
 'mextraccion_autoservicio',
 'mpasivos_margen',
 'mrentabilidad',
 'mrentabilidad_annual',
 'mtarjeta_visa_consumo',
 'mtransferencias_recibidas',
 'mttarjeta_visa_debitos_automaticos',
 'thomebanking',
 'tmobile_app'}

In [33]:
# Parece que visa_fultimo_cierre y master_fultimo_cierre tiene sospechosos ceros en junio. Elimino las columnas
columnas_atipicas = [
    'Master_mfinanciacion_limite', 
    'Visa_mfinanciacion_limite', 
    'Visa_Finiciomora',
    'Master_Finiciomora', 
    'Master_fultimo_cierre', 
    'Visa_fultimo_cierre',
]
data.drop(columns=columnas_atipicas, inplace=True)
print (f"Cantidad de columnas restantes: {len(data.columns)}")

Cantidad de columnas restantes: 149


#### Valores duplicados.

In [34]:
#a. Verifico.
print(f"Data tiene {data.duplicated().sum()} filas duplicadas")


Data tiene 0 filas duplicadas


## C. Data Drifting.

In [35]:
def psi(expected, actual, buckets=10):

    def psi_formula(expected_prop, actual_prop):
        result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
        return result

    expected_not_null = expected.dropna()
    actual_not_null = actual.dropna()

    bin_edges = pd.qcut(expected_not_null, q=buckets, duplicates='drop').unique().dropna()

    bin_edges2 = [edge.left for edge in bin_edges] + [edge.right for edge in bin_edges]
    breakpoints = sorted(list(set(bin_edges2)))

    expected_counts, _ = np.histogram(expected_not_null, bins=breakpoints)
    actual_counts, _ = np.histogram(actual_not_null, bins=breakpoints)

    expected_prop = expected_counts / len(expected_not_null)
    actual_prop = actual_counts / len(actual_not_null)

    psi_not_null = psi_formula(expected_prop, actual_prop).sum()

    psi_null = 0

    if expected.isnull().sum() > 0 and actual.isnull().sum() > 0 :
      expected_null_percentage = expected.isnull().mean()
      actual_null_percentage = actual.isnull().mean()
      psi_null = psi_formula(expected_null_percentage, actual_null_percentage)

    return psi_not_null + psi_null


psi_results = []

# Obtener y ordenar los meses únicos
meses = sorted(data['foto_mes'].unique())

# Recorrer los meses de dos en dos (actual y siguiente)
for i in range(len(meses) - 1):
    mes_actual = meses[i]
    mes_siguiente = meses[i + 1]
    print(f"Mes actual: {mes_actual}, Mes siguiente: {mes_siguiente}")
    train_data = data[data['foto_mes'] == mes_actual]
    score_data = data[data['foto_mes'] == mes_siguiente]
    for column in data.columns:
      if column not in ['foto_mes','numero_de_cliente','clase_ternaria']:
        train_variable = train_data[column]
        score_variable = score_data[column]
        psi_value = psi(train_variable, score_variable)
        psi_results.append({'mes': mes_siguiente, 'feature': column, 'psi': psi_value})

    psi_df = pd.DataFrame(psi_results)
    psi_df = psi_df.sort_values('psi', ascending=False)

Mes actual: 201901, Mes siguiente: 201902


/tmp/ipykernel_2178/2443564196.py:4: RuntimeWarning: divide by zero encountered in log
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)


Mes actual: 201902, Mes siguiente: 201903


/tmp/ipykernel_2178/2443564196.py:4: RuntimeWarning: divide by zero encountered in log
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)


Mes actual: 201903, Mes siguiente: 201904


...

/tmp/ipykernel_2178/2443564196.py:4: RuntimeWarning: divide by zero encountered in log
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)


Mes actual: 202101, Mes siguiente: 202102
Mes actual: 202102, Mes siguiente: 202103


/tmp/ipykernel_2178/2443564196.py:4: RuntimeWarning: divide by zero encountered in log
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)


Mes actual: 202103, Mes siguiente: 202104
Mes actual: 202104, Mes siguiente: 202105
Mes actual: 202105, Mes siguiente: 202106
Mes actual: 202106, Mes siguiente: 202107
Mes actual: 202107, Mes siguiente: 202108
Mes actual: 202108, Mes siguiente: 202109


/tmp/ipykernel_2178/2443564196.py:4: RuntimeWarning: divide by zero encountered in log
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)


In [36]:

#2. Observamos variables con PSI >= 0.1 (casos a atender).
psi_df[psi_df["psi"] > 0.1][['mes', "feature"]]

,mes,feature
17,201902,mcaja_ahorro_adicional
1173,201910,mrentabilidad
1239,201910,ccomisiones_otras
4543,202109,mcaja_ahorro_adicional
2353,202006,mcaja_ahorro_adicional
...,...,...
2474,202006,Visa_mpagado
3408,202101,mpayroll
780,201907,mpayroll
3722,202103,mcomisiones_otras


In [37]:
#3. Elimino las columnas con un PSI alto.
cols_eliminar = list(psi_df[psi_df["psi"] > 0.1]["feature"])
#data.drop(columns=cols_eliminar,inplace=True)

#train_data.drop(columns=cols_eliminar,inplace=True)
#score_data.drop(columns=cols_eliminar,inplace=True)
print (f"Cantidad de columnas restantes: {len(data.columns)}")

Cantidad de columnas restantes: 149


In [39]:
# eliminar selectivamente variables
data = data.loc[:, ~data.columns.str.contains('prestamos_personales', case=False)]
data = data.loc[:, ~data.columns.str.contains('ultimo_cierre', case=False)]
print (f"Cantidad de columnas restantes: {len(data.columns)}")

Cantidad de columnas restantes: 147


## Exportación.

In [40]:
#1. Exportación propiamente dicha.
dataset_output=os.path.join(dataset_path, output_file)
if dataset_output.endswith('.gz'):
  data.to_parquet(dataset_output,index=False, compression='gzip')
else:
  data.to_parquet(dataset_output,index=False)

In [41]:
print (dataset_output)

~/buckets/b1/datasets/k3__p__.parquet.gz


In [42]:
print (f"Cantidad de columnas restantes: {len(data.columns)}")

Cantidad de columnas restantes: 147
